# Sistema de recomendación


## ¿Qué son los sistemas de recomendación?

Los sistemas de recomendación son algoritmos diseñados para sugerir elementos (productos, películas, música, etc.) a los usuarios en función de sus preferencias y comportamientos anteriores. Estos sistemas se han vuelto fundamentales en diversas aplicaciones modernas, como tiendas en línea, plataformas de streaming y redes sociales.

### Tipos de Sistemas de Recomendación

1. **Filtrado colaborativo basado en usuarios**:
   Este enfoque recomienda productos a un usuario basándose en las preferencias de usuarios similares. La idea es que si un grupo de usuarios tiene gustos parecidos, los elementos que un usuario ha valorado positivamente pueden ser recomendados a otros usuarios similares.

2. **Filtrado colaborativo basado en ítems**:
   En este enfoque, las recomendaciones se hacen basadas en la similitud entre productos. Si un usuario ha valorado positivamente un ítem, se le recomendarán ítems similares.

3. **Modelos basados en contenido**:
   Se recomiendan ítems similares a aquellos que un usuario ya ha mostrado interés, basándose en las características de los ítems, como género, autor, director, etc.

4. **Sistemas híbridos**:
   Combinan diferentes enfoques, como el filtrado colaborativo y el contenido, para mejorar la precisión de las recomendaciones.

## ¿Por qué son importantes?

Los sistemas de recomendación son cruciales porque mejoran la experiencia del usuario al reducir la sobrecarga de información y personalizar la oferta de contenido o productos. Además, ayudan a las empresas a aumentar sus ventas o retener a los usuarios al ofrecer recomendaciones más precisas y relevantes.



In [1]:
### EJEMPLO SISTEMA DE RECOMENDACIÓN
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

In [2]:
# Cargar el dataset de MovieLens
url = "https://files.grouplens.org/datasets/movielens/ml-100k/u.data"
columns = ['user_id', 'item_id', 'rating', 'timestamp']
data = pd.read_csv(url, sep='\t', names=columns)

In [4]:
# data['item_id'] unicos
print(data['item_id'].nunique())

1682


In [5]:
# data['user_id'] unicos
print(data['user_id'].nunique())

943


In [6]:
# valores unicos en data['rating']
print(data['rating'].unique())

[3 1 2 4 5]


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   user_id    100000 non-null  int64
 1   item_id    100000 non-null  int64
 2   rating     100000 non-null  int64
 3   timestamp  100000 non-null  int64
dtypes: int64(4)
memory usage: 3.1 MB


In [8]:
# contar valores nan
print(data.isnull().sum())

user_id      0
item_id      0
rating       0
timestamp    0
dtype: int64


In [9]:
# Preprocesar los datos
data = data.drop('timestamp', axis=1)

In [10]:
data.head()

,user_id,item_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [11]:
# Crear la matriz usuario-item
user_item_matrix = data.pivot_table(index='user_id', columns='item_id', values='rating')

In [12]:
user_item_matrix

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Missing value
# Opción 1: promedio de data['rating']
# data['rating'].fillna(data['rating'].mean(), inplace=True)
prom = data['rating'].mean()
print(prom)

3.52986


In [14]:
# Rellenar los valores NaN con ceros (puedes optar por otra estrategia, como promedios)
user_item_matrix = user_item_matrix.fillna(0)

In [15]:
user_item_matrix

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# Calcular la similitud entre usuarios utilizando la similitud del coseno
user_similarity = cosine_similarity(user_item_matrix)

In [17]:
user_similarity

array([[1.        , 0.16693098, 0.04745954, ..., 0.14861694, 0.17950788,
        0.39817474],
       [0.16693098, 1.        , 0.11059132, ..., 0.16148478, 0.17226781,
        0.10579788],
       [0.04745954, 0.11059132, 1.        , ..., 0.10124256, 0.13341615,
        0.02655587],
       ...,
       [0.14861694, 0.16148478, 0.10124256, ..., 1.        , 0.1016418 ,
        0.09511958],
       [0.17950788, 0.17226781, 0.13341615, ..., 0.1016418 , 1.        ,
        0.18246466],
       [0.39817474, 0.10579788, 0.02655587, ..., 0.09511958, 0.18246466,
        1.        ]])

In [18]:
# Convertir la matriz de similitud en un DataFrame para facilitar el manejo
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

In [19]:
user_similarity_df

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.166931,0.047460,0.064358,0.378475,0.430239,0.440367,0.319072,0.078138,0.376544,...,0.369527,0.119482,0.274876,0.189705,0.197326,0.118095,0.314072,0.148617,0.179508,0.398175
2,0.166931,1.000000,0.110591,0.178121,0.072979,0.245843,0.107328,0.103344,0.161048,0.159862,...,0.156986,0.307942,0.358789,0.424046,0.319889,0.228583,0.226790,0.161485,0.172268,0.105798
3,0.047460,0.110591,1.000000,0.344151,0.021245,0.072415,0.066137,0.083060,0.061040,0.065151,...,0.031875,0.042753,0.163829,0.069038,0.124245,0.026271,0.161890,0.101243,0.133416,0.026556
4,0.064358,0.178121,0.344151,1.000000,0.031804,0.068044,0.091230,0.188060,0.101284,0.060859,...,0.052107,0.036784,0.133115,0.193471,0.146058,0.030138,0.196858,0.152041,0.170086,0.058752
5,0.378475,0.072979,0.021245,0.031804,1.000000,0.237286,0.373600,0.248930,0.056847,0.201427,...,0.338794,0.080580,0.094924,0.079779,0.148607,0.071459,0.239955,0.139595,0.152497,0.313941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.118095,0.228583,0.026271,0.030138,0.071459,0.111852,0.107027,0.095898,0.039852,0.071460,...,0.066039,0.431154,0.258021,0.226449,0.432666,1.000000,0.087687,0.180029,0.043264,0.144250
940,0.314072,0.226790,0.161890,0.196858,0.239955,0.352449,0.329925,0.246883,0.120495,0.342961,...,0.327153,0.107024,0.187536,0.181317,0.175158,0.087687,1.000000,0.145152,0.261376,0.241028
941,0.148617,0.161485,0.101243,0.152041,0.139595,0.144446,0.059993,0.146145,0.143245,0.090305,...,0.046952,0.203301,0.288318,0.234211,0.313400,0.180029,0.145152,1.000000,0.101642,0.095120


In [20]:
# Función para hacer recomendaciones basadas en la similitud de usuarios
def recommend_movies(user_id, n_recommendations=5):
    # Obtener la similitud del usuario dado con otros usuarios
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)

    # Obtener las películas que el usuario aún no ha calificado
    user_ratings = user_item_matrix.loc[user_id]
    unrated_movies = user_ratings[user_ratings == 0].index

    # Hacer la media ponderada de las calificaciones de los usuarios similares para las películas no calificadas
    recommendations = {}
    for movie in unrated_movies:
        similar_users_ratings = user_item_matrix.loc[similar_users.index, movie]
        weighted_sum = np.dot(similar_users.values, similar_users_ratings.values)
        recommendations[movie] = weighted_sum

    # Ordenar las recomendaciones y devolver las mejores
    recommended_movies = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)[:n_recommendations]

    return recommended_movies


In [21]:
# Ejemplo: Hacer recomendaciones para el usuario con ID 1
recommendations = recommend_movies(1)
print(f"Recomendaciones para el usuario 1: {recommendations}")

Recomendaciones para el usuario 1: [(318, 472.0552903688434), (423, 433.97200624206374), (357, 415.62717779869877), (286, 404.1028993953055), (288, 393.35046489592855)]


In [22]:
# Ejemplo: Hacer recomendaciones para el usuario con ID 900
recommendations = recommend_movies(900)
print(f"Recomendaciones para el usuario 1: {recommendations}")

Recomendaciones para el usuario 1: [(50, 375.9570930792481), (181, 297.84595512897215), (127, 277.1682092005474), (1, 274.9492051359621), (174, 268.7652789578263)]


# Bases de Datos para Practicar Sistemas de Recomendación

A continuación, te dejo una lista de bases de datos públicas que puedes utilizar para practicar y desarrollar sistemas de recomendación. Estas bases de datos cubren diferentes dominios, como películas, música, libros y productos.

## 1. [MovieLens](https://grouplens.org/datasets/movielens/)
**Descripción**:
- MovieLens es una de las bases de datos más populares para sistemas de recomendación. Contiene calificaciones de películas realizadas por usuarios.
- Disponible en diferentes tamaños:
  - **100k**: 100,000 calificaciones por 943 usuarios sobre 1,682 películas.
  - **1M**: 1 millón de calificaciones por 6,000 usuarios sobre 4,000 películas.
  - **20M**: 20 millones de calificaciones de películas.

**Aplicación**: Recomendación de películas.

## 2. [Jester](http://eigentaste.berkeley.edu/dataset/)
**Descripción**:
- Este dataset contiene calificaciones de 100 chistes por parte de más de 73,000 usuarios.
- Ideal para sistemas de recomendación de entretenimiento o basados en preferencias humorísticas.

**Aplicación**: Recomendación de chistes o contenido humorístico.

## 3. [Amazon Product Data](http://jmcauley.ucsd.edu/data/amazon/)
**Descripción**:
- Contiene reseñas de productos en Amazon, organizadas por categoría (libros, electrónica, ropa, etc.).
- Incluye información sobre calificaciones, texto de las reseñas y relaciones entre productos.

**Aplicación**: Recomendación de productos, análisis de sentimientos, o sistemas basados en reseñas.

## 4. [Book-Crossing Dataset](https://www.kaggle.com/datasets/somnambwl/bookcrossing-dataset)
**Descripción**:
- Dataset de recomendaciones de libros que incluye 1.1 millones de calificaciones de 278,000 usuarios sobre 271,000 libros.
- Incluye información sobre los usuarios, como su edad y ubicación.

**Aplicación**: Recomendación de libros.

## 5. [Last.fm Dataset](http://millionsongdataset.com/lastfm/)
**Descripción**:
- Contiene interacciones entre usuarios y música (pistas reproducidas, artistas, etc.) de la plataforma Last.fm.
- El dataset incluye datos de 1,000 usuarios con más de 1 millón de registros de reproducciones.

**Aplicación**: Recomendación de música y artistas.

## 6. [Netflix Prize Dataset](https://www.kaggle.com/datasets/netflix-inc/netflix-prize-data)
**Descripción**:
- Dataset utilizado para la competencia **Netflix Prize**. Contiene millones de calificaciones de usuarios sobre películas.
- Aunque la competencia ya terminó, este dataset sigue siendo muy útil para la investigación en recomendaciones.

**Aplicación**: Recomendación de películas y contenido multimedia.

## 7. [Yelp Dataset](https://www.yelp.com/dataset)
**Descripción**:
- Contiene reseñas y calificaciones de empresas locales (restaurantes, tiendas, etc.) en diferentes ciudades.
- Además de calificaciones, incluye información de los usuarios y las empresas, como categorías, ubicación y horarios.

**Aplicación**: Recomendación de negocios y servicios locales.

## 8. [Goodreads Book Reviews](https://www.kaggle.com/datasets/pypiahmad/goodreads-book-reviews1)
**Descripción**:
- Dataset que contiene reseñas y calificaciones de libros de **Goodreads**.
- Se puede utilizar para recomendar libros basándose en las calificaciones y el texto de las reseñas.

**Aplicación**: Recomendación de libros y análisis de reseñas.

## 9. [Spotify Million Playlist Dataset](https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge)
**Descripción**:
- Dataset de Spotify que contiene 1 millón de listas de reproducción creadas por usuarios. Cada lista de reproducción incluye una serie de canciones seleccionadas por los usuarios.

**Aplicación**: Recomendación de música y generación de listas de reproducción.

## 10. [IMDb Dataset](https://www.imdb.com/interfaces/)
**Descripción**:
- Dataset proporcionado por IMDb, que incluye información sobre películas, programas de televisión, actores, directores, y mucho más.
- Aunque no contiene calificaciones directas, se puede usar para sistemas de recomendación basados en contenido (género, director, etc.).

**Aplicación**: Recomendación de películas y series basadas en metadatos.

---

## Recomendaciones adicionales

### [Kaggle](https://www.kaggle.com/datasets)
Kaggle es una excelente fuente de datasets para recomendaciones y aprendizaje automático. Puedes encontrar datasets etiquetados como **Recommender Systems** o explorar cualquier dataset para crear tus propios sistemas de recomendación.

---

### Consideraciones al elegir una base de datos:
1. **Tamaño del Dataset**: Algunos datasets son muy grandes, por lo que debes asegurarte de tener suficiente capacidad de procesamiento.
2. **Dominio**: Elige un dataset que esté alineado con el tipo de sistema de recomendación que quieras construir (películas, libros, música, productos, etc.).
3. **Información Adicional**: Algunas bases de datos ofrecen información adicional como texto de reseñas, categorías de productos, géneros de películas, etc., lo que puede ser útil para implementar **modelos híbridos** o mejorar el rendimiento.



In [ ]:
# Leer
# https://medium.com/@EmiLabsTech/data-privacy-the-netflix-prize-competition-84330d01cc34

# ¿Por qué usar scikit-surprise?

## ¿Qué es scikit-surprise?

**`scikit-surprise`** es una biblioteca de Python diseñada específicamente para implementar y evaluar sistemas de recomendación. A diferencia de otras herramientas generales de machine learning, **surprise** está enfocada en tareas específicas de recomendación y simplifica mucho el manejo de datos de calificaciones.

### Ventajas de usar `scikit-surprise`:

1. **Facilidad de uso**:
   `scikit-surprise` permite implementar algoritmos complejos de sistemas de recomendación con muy pocas líneas de código. Los algoritmos más utilizados, como **SVD** (Singular Value Decomposition) o **KNN** (K Nearest Neighbors), están ya integrados.

2. **Datasets incorporados**:
   La biblioteca proporciona acceso a datasets populares de calificaciones, como **MovieLens** o **Jester**, lo que facilita la experimentación y aprendizaje sin la necesidad de buscar y preparar datos externos.

3. **Evaluación y validación**:
   `scikit-surprise` proporciona herramientas para realizar validaciones cruzadas y evaluar el rendimiento de los modelos, lo que es clave para comparar la efectividad de diferentes enfoques de recomendación.

4. **Extensibilidad**:
   Puedes implementar tus propios algoritmos de recomendación si los existentes no cumplen con tus necesidades. `scikit-surprise` te permite definir modelos personalizados con facilidad.

5. **Soporte para múltiples algoritmos**:
   La biblioteca incluye soporte para diversos algoritmos de filtrado colaborativo, tanto basado en usuarios como en ítems, así como modelos basados en matrices factorizadas, como **SVD** y **NMF**.

### Algoritmos disponibles en scikit-surprise:

- **SVD**: Uno de los más utilizados, basado en la factorización de matrices.
- **KNNBasic**: Filtrado colaborativo con vecinos más cercanos.
- **Slope One**: Un algoritmo simple y efectivo para sistemas de recomendación.
- **Co-Clustering**: Algoritmo que agrupa simultáneamente usuarios y elementos para recomendaciones.

### Ejemplo básico:

```python
from surprise import Dataset, SVD
from surprise.model_selection import cross_validate

# Cargar el dataset MovieLens 100k directamente desde Surprise
data = Dataset.load_builtin('ml-100k')

# Usar el algoritmo SVD
algo = SVD()

# Validar el modelo con validación cruzada
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


In [23]:
# Lo primero es instalar
%pip install scikit-surprise gcsfs --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# ver
# https://surprise.readthedocs.io/en/stable/_modules/surprise/dataset.html#Dataset

# Bases de Datos en `scikit-surprise`

`scikit-surprise` incluye varios datasets incorporados que puedes cargar fácilmente para practicar la creación de sistemas de recomendación. Estos datasets están centrados principalmente en calificaciones de usuarios sobre distintos tipos de ítems como películas, libros, chistes, etc.

## 1. MovieLens 100k

**Descripción**:
- Dataset de 100,000 calificaciones de 943 usuarios sobre 1,682 películas. Es uno de los datasets más utilizados para sistemas de recomendación.

**Tamaño**: 100,000 calificaciones

**Aplicación**: Recomendación de películas.

**Cómo cargarlo**:

```python
from surprise import Dataset
data = Dataset.load_builtin('ml-100k')


## 2. MovieLens 1M

**Descripción**:
- Una versión más grande del dataset de MovieLens que contiene 1 millón de calificaciones realizadas por 6,000 usuarios sobre 4,000 películas.

**Tamaño**: 1,000,000 calificaciones

**Aplicación**: Recomendación de películas a mayor escala.

**Cómo cargarlo**:

```python
from surprise import Dataset
data = Dataset.load_builtin('ml-1m')


## 3. Jester Dataset

**Descripción**:
- Este dataset contiene calificaciones de 100 chistes por parte de más de 73,000 usuarios. Ideal para sistemas de recomendación relacionados con contenido humorístico.

**Tamaño**: Más de 1.7 millones de calificaciones de chistes

**Aplicación**: Recomendación de chistes o contenido humorístico.

**Cómo cargarlo**:

```python
from surprise import Dataset
data = Dataset.load_builtin('jester')


## 4. Book-Crossing Dataset

**Descripción**:
- Un dataset de recomendaciones de libros que incluye 1.1 millones de calificaciones realizadas por 278,000 usuarios sobre 271,000 libros.

**Tamaño**: 1,149,780 calificaciones

**Aplicación**: Recomendación de libros.

**Cómo cargarlo**:

```python
from surprise import Dataset
data = Dataset.load_builtin('bx')


## 5. Netflix Prize Dataset

**Descripción**:
- Este dataset es una muestra de las calificaciones de películas utilizadas en la famosa competencia Netflix Prize, donde se buscaba mejorar las recomendaciones de películas.

**Tamaño**: Desconocido (enorme)

**Aplicación**: Recomendación de películas y contenido multimedia.

**Cómo cargarlo**:

```python
from surprise import Dataset
data = Dataset.load_builtin('netflix')



# SVD en `scikit-surprise`

El **SVD** (Singular Value Decomposition o Descomposición en Valores Singulares) es uno de los algoritmos más utilizados para implementar **filtrado colaborativo basado en descomposición matricial**. Este enfoque predice las calificaciones que un usuario podría dar a ítems (como películas o productos) basándose en las calificaciones anteriores, tanto del usuario como de otros usuarios similares.

## ¿Cómo funciona el SVD?

El algoritmo SVD descompone la matriz de calificaciones en tres matrices:
- Una matriz que representa los usuarios.
- Una matriz que representa los ítems.
- Una matriz diagonal que contiene los valores singulares, que indican la importancia de cada componente.

Matemáticamente, si tenemos una matriz de calificaciones **R** de tamaño \(m \times n\) (donde \(m\) son los usuarios y \(n\) son los ítems), el SVD descompone esa matriz de la siguiente manera:

$$
R \approx U \Sigma V^T
$$

Donde:
- **U** es una matriz \(m \times k\) que representa a los usuarios en un espacio latente.
- **\Sigma** es una matriz diagonal \(k \times k\) que contiene los valores singulares.
- **V^T** es una matriz \(k \times n\) que representa los ítems en ese mismo espacio latente.

El valor de \(k\) es un hiperparámetro que define el número de dimensiones en el espacio latente.

## SVD en `scikit-surprise`

En `scikit-surprise`, el algoritmo SVD es una implementación simplificada del enfoque utilizado por el equipo ganador del **Netflix Prize**. Se basa en descomponer la matriz de calificaciones en componentes de usuario e ítem, y luego utilizar estos componentes para hacer predicciones de calificaciones faltantes.

### Parámetros importantes:

- **n_factors**: Número de factores latentes. Controla la complejidad del modelo.
- **n_epochs**: Número de iteraciones del algoritmo de optimización.
- **lr_all**: Tasa de aprendizaje utilizada en el descenso de gradiente.
- **reg_all**: Parámetro de regularización que ayuda a controlar el sobreajuste.

## Ventajas del SVD:

1. **Reducción de dimensionalidad**: Permite reducir el número de características a unas pocas dimensiones latentes, mejorando la generalización del modelo.
  
2. **Predicciones robustas**: SVD es eficaz para predecir calificaciones incluso cuando las matrices de calificaciones son muy dispersas (con muchas entradas faltantes).

3. **Probado en la práctica**: Fue uno de los enfoques utilizados por los ganadores del **Netflix Prize**, demostrando su efectividad en escenarios de recomendación a gran escala.

### ¿Cuándo NO usar SVD?

- **Datos demasiado pequeños**: Si tienes pocos datos, el modelo puede no generalizar bien, ya que el algoritmo necesita suficiente información para identificar la estructura latente subyacente.
- **Filtrado colaborativo basado en contenido**: Si las características de los ítems (géneros de películas, autores, etc.) son importantes, el SVD puede no ser la mejor opción, ya que se basa únicamente en las interacciones usuario-ítem.

---

## Ejemplo de uso de SVD en `scikit-surprise`

A continuación se muestra cómo usar el algoritmo SVD en un conjunto de datos de calificaciones de películas (MovieLens 100k) utilizando la biblioteca `scikit-surprise`:

```python
# Importar las librerías necesarias
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy

# Cargar el dataset de MovieLens 100k
data = Dataset.load_builtin('ml-100k')

# Dividir los datos en entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.25)

# Usar el algoritmo SVD
algo = SVD()

# Entrenar el modelo en el set de entrenamiento
algo.fit(trainset)

# Predecir las calificaciones en el set de prueba
predictions = algo.test(testset)

# Evaluar el rendimiento con el error RMSE
accuracy.rmse(predictions)


In [24]:
### EJEMPLO DE EJERCICIO

from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [25]:
# Cargar el dataset MovieLens-100k
data = Dataset.load_builtin('ml-100k')

# Dividir el dataset en conjunto de entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=.25)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [26]:
# Crear el algoritmo SVD
svd = SVD()

# Entrenar el algoritmo con el conjunto de entrenamiento
svd.fit(trainset)

In [27]:
# Hacer predicciones en el conjunto de prueba
predictions = svd.test(testset)

In [28]:
# Calcular el RMSE (Root Mean Squared Error)
rmse = accuracy.rmse(predictions)
print(f"RMSE: {rmse}")

RMSE: 0.9484
RMSE: 0.9484154519513754


In [29]:
# Hacer algunas recomendaciones
def get_top_n(predictions, n=10):
    top_n = {}
    for uid, iid, true_r, est, _ in predictions:
        if uid not in top_n:
            top_n[uid] = []
        top_n[uid].append((iid, est))

    # Ordenar las predicciones para cada usuario y obtener las top n
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n



In [30]:
top_n = get_top_n(predictions, n=5)

In [31]:
# Imprimir las top 5 recomendaciones para el usuario 1
print("\nTop 5 recomendaciones para el usuario 1:")
for iid, rating in top_n['1']:
    print(f"Película {iid}: Calificación estimada {rating:.2f}")


Top 5 recomendaciones para el usuario 1:
Película 100: Calificación estimada 4.59
Película 191: Calificación estimada 4.53
Película 189: Calificación estimada 4.35
Película 64: Calificación estimada 4.30
Película 12: Calificación estimada 4.29


# KNNBasic en Surprise

## Introducción

`KNNBasic` es un algoritmo de recomendación basado en la técnica de K-Nearest Neighbors (KNN). Este enfoque se utiliza comúnmente para sistemas de recomendación que buscan encontrar elementos similares en función de las calificaciones dadas por los usuarios. `KNNBasic` en la biblioteca Surprise permite crear recomendaciones mediante el uso de distancias entre usuarios o ítems.

## ¿Cómo funciona?

El algoritmo de KNN funciona identificando los **K** vecinos más cercanos de un usuario o ítem dado. Hay dos maneras principales de aplicar KNN:

1. **Filtrado colaborativo basado en usuarios**: Recomendaciones basadas en usuarios similares.
2. **Filtrado colaborativo basado en ítems**: Recomendaciones basadas en ítems similares.

### Pasos Generales

1. **Cargar los datos**: Utilizar un conjunto de datos que contenga interacciones entre usuarios y productos.
2. **Crear el modelo KNN**: Especificar la métrica de similitud y el número de vecinos a considerar.
3. **Entrenar el modelo**: Ajustar el modelo a los datos.
4. **Hacer recomendaciones**: Predecir las calificaciones para elementos no calificados por el usuario.

## Ejemplo de Código

A continuación se presenta un ejemplo básico que utiliza `KNNBasic` de la biblioteca Surprise para hacer recomendaciones basadas en usuarios:

```python
# Importar las bibliotecas necesarias
from surprise import Dataset, Reader
from surprise import KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy

# Cargar el conjunto de datos de MovieLens
data = Dataset.load_builtin('ml-100k')
reader = Reader(line_format='user item rating timestamp', sep='\t')

# Cargar los datos
data = Dataset.load_from_file('ml-100k/u.data', reader=reader)

# Dividir los datos en conjunto de entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.25)

# Crear el modelo KNN con similaridad basada en coseno
sim_options = {
    'name': 'cosine',
    'user_based': True  # Utilizar similitud entre usuarios
}
knn = KNNBasic(sim_options=sim_options)

# Entrenar el modelo
knn.fit(trainset)

# Hacer predicciones
predictions = knn.test(testset)

# Evaluar el modelo
rmse = accuracy.rmse(predictions)
print(f'RMSE: {rmse}')


In [32]:
from surprise import KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from collections import defaultdict

# Cargar el dataset MovieLens-100k
data = Dataset.load_builtin('ml-100k')

# Dividir el dataset en conjunto de entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=.25)

# Crear el algoritmo KNNBasic
# Usamos similitud basada en coseno y k=40 vecinos
modelo = KNNBasic(k=40, sim_options={'name': 'cosine', 'user_based': True})

# Entrenar el algoritmo con el conjunto de entrenamiento
modelo.fit(trainset)

# Hacer predicciones en el conjunto de prueba
predictions = modelo.test(testset)

# Calcular el RMSE (Root Mean Squared Error)
rmse = accuracy.rmse(predictions)
print(f"RMSE: {rmse}")

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0212
RMSE: 1.0212189853840217


In [33]:
# Mostrar algunas predicciones individuales
print("\nAlgunas predicciones individuales:")
for i, prediction in enumerate(predictions[:5]):
    print(f"Usuario: {prediction.uid}, Película: {prediction.iid}, "
          f"Calificación real: {prediction.r_ui}, Calificación estimada: {prediction.est:.2f}")



Algunas predicciones individuales:
Usuario: 717, Película: 405, Calificación real: 3.0, Calificación estimada: 3.47
Usuario: 259, Película: 294, Calificación real: 3.0, Calificación estimada: 3.33
Usuario: 887, Película: 409, Calificación real: 4.0, Calificación estimada: 3.20
Usuario: 280, Película: 155, Calificación real: 5.0, Calificación estimada: 3.28
Usuario: 200, Película: 258, Calificación real: 4.0, Calificación estimada: 3.95


In [34]:
# Función para obtener las top N recomendaciones
def get_top_n(predictions, n=5):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Ordenar las predicciones para cada usuario y obtener las top n
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return dict(top_n)

# Obtener las top 5 recomendaciones para todos los usuarios
top_n = get_top_n(predictions, n=5)

# Mostrar las top 5 recomendaciones para el usuario 1
print("\nTop 5 recomendaciones para el usuario 1:")
for iid, rating in top_n['1']:
    print(f"Película {iid}: Calificación estimada {rating:.2f}")




Top 5 recomendaciones para el usuario 1:
Película 127: Calificación estimada 4.55
Película 114: Calificación estimada 4.47
Película 174: Calificación estimada 4.45
Película 170: Calificación estimada 4.33
Película 272: Calificación estimada 4.32


In [35]:
# Mostrar todas las predicciones para el usuario 1
print("\nTodas las predicciones para el usuario 1:")
user_1_predictions = [pred for pred in predictions if pred.uid == '1']
for prediction in sorted(user_1_predictions, key=lambda x: x.est, reverse=True)[:10]:
    print(f"Película {prediction.iid}: Calificación estimada {prediction.est:.2f}")


Todas las predicciones para el usuario 1:
Película 127: Calificación estimada 4.55
Película 114: Calificación estimada 4.47
Película 174: Calificación estimada 4.45
Película 170: Calificación estimada 4.33
Película 272: Calificación estimada 4.32
Película 9: Calificación estimada 4.30
Película 242: Calificación estimada 4.28
Película 183: Calificación estimada 4.25
Película 83: Calificación estimada 4.20
Película 190: Calificación estimada 4.20


## Ejemplo con nombre de películas

In [36]:
import os
import urllib.request
import zipfile
from surprise import KNNBasic, Dataset, Reader
from surprise import accuracy
from surprise.model_selection import train_test_split
from collections import defaultdict
import pandas as pd

# Función para descargar y extraer el dataset MovieLens-100k
def download_movielens_100k():
    url = "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
    zip_path = "ml-100k.zip"
    extract_path = "."

    if not os.path.exists("ml-100k"):
        print("Descargando MovieLens-100k dataset...")
        urllib.request.urlretrieve(url, zip_path)

        print("Extrayendo archivos...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)

        os.remove(zip_path)
        print("Dataset descargado y extraído exitosamente.")
    else:
        print("El dataset MovieLens-100k ya existe.")

# Descargar el dataset si no existe
download_movielens_100k()

# Cargar los metadatos de las películas
movies_df = pd.read_csv('ml-100k/u.item', sep='|', encoding='latin-1', header=None,
                        names=['movie_id', 'title', 'release_date', 'video_release_date',
                               'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation',
                               'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
                               'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
                               'Thriller', 'War', 'Western'])

# Crear un diccionario para mapear id de película a título
movie_id_to_title = dict(zip(movies_df['movie_id'], movies_df['title']))

# Cargar el dataset de calificaciones
file_path = os.path.expanduser('ml-100k/u.data')
reader = Reader(line_format='user item rating timestamp', sep='\t')
data = Dataset.load_from_file(file_path, reader=reader)

# Dividir el dataset en conjunto de entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=.25)

# Crear el algoritmo KNNBasic
modelo = KNNBasic(k=40, sim_options={'name': 'cosine', 'user_based': True})

# Entrenar el algoritmo con el conjunto de entrenamiento
modelo.fit(trainset)

# Hacer predicciones en el conjunto de prueba
predictions = modelo.test(testset)

# Calcular el RMSE (Root Mean Squared Error)
rmse = accuracy.rmse(predictions)
print(f"RMSE: {rmse}")

# Función para obtener las top N recomendaciones
def get_top_n(predictions, n=5):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Ordenar las predicciones para cada usuario y obtener las top n
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return dict(top_n)

# Obtener las top 5 recomendaciones para todos los usuarios
top_n = get_top_n(predictions, n=5)

# Mostrar las top 5 recomendaciones para el usuario 1
print("\nTop 5 recomendaciones para el usuario 1:")
for iid, rating in top_n['1']:
    print(f"Película: {movie_id_to_title[int(iid)]}, Calificación estimada: {rating:.2f}")

# Mostrar todas las predicciones para el usuario 1
print("\nTodas las predicciones para el usuario 1:")
user_1_predictions = [pred for pred in predictions if pred.uid == '1']
for prediction in sorted(user_1_predictions, key=lambda x: x.est, reverse=True)[:10]:
    print(f"Película: {movie_id_to_title[int(prediction.iid)]}, "
          f"Calificación estimada: {prediction.est:.2f}")

Descargando MovieLens-100k dataset...
Extrayendo archivos...
Dataset descargado y extraído exitosamente.
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0156
RMSE: 1.0155620341809661

Top 5 recomendaciones para el usuario 1:
Película: Usual Suspects, The (1995), Calificación estimada: 4.48
Película: Citizen Kane (1941), Calificación estimada: 4.48
Película: Dead Man Walking (1995), Calificación estimada: 4.33
Película: Room with a View, A (1986), Calificación estimada: 4.30
Película: Three Colors: Red (1994), Calificación estimada: 4.25

Todas las predicciones para el usuario 1:
Película: Usual Suspects, The (1995), Calificación estimada: 4.48
Película: Citizen Kane (1941), Calificación estimada: 4.48
Película: Dead Man Walking (1995), Calificación estimada: 4.33
Película: Room with a View, A (1986), Calificación estimada: 4.30
Película: Three Colors: Red (1994), Calificación estimada: 4.25
Película: Priest (1994), Calificación estimada: 4.23
Películ

In [37]:
# Mostrar todas las predicciones para el usuario 1
print("\nTodas las predicciones para el usuario 900:")
user_1_predictions = [pred for pred in predictions if pred.uid == '900']
for prediction in sorted(user_1_predictions, key=lambda x: x.est, reverse=True)[:10]:
    print(f"Película: {movie_id_to_title[int(prediction.iid)]}, "
          f"Calificación estimada: {prediction.est:.2f}")


Todas las predicciones para el usuario 900:
Película: Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963), Calificación estimada: 4.55
Película: Casablanca (1942), Calificación estimada: 4.43
Película: Schindler's List (1993), Calificación estimada: 4.35
Película: Alien (1979), Calificación estimada: 4.17
Película: Shining, The (1980), Calificación estimada: 3.85
Película: American in Paris, An (1951), Calificación estimada: 3.71
Película: Scream (1996), Calificación estimada: 3.45
Película: City Hall (1996), Calificación estimada: 3.20
Película: Mission: Impossible (1996), Calificación estimada: 3.15
Película: Crash (1996), Calificación estimada: 2.67


In [38]:
data

[Datasets for recommender Systems](https://github.com/caserec/Datasets-for-Recommender-Systems )

In [42]:
%%javascript
console.log('Hola mundo!')

<IPython.core.display.Javascript object>

In [43]:
%%html
<!-- Crear tres botones en HTML -->
<button id="btn1">Botón 1</button>
<button id="btn2">Botón 2</button>
<button id="btn3">Botón 3</button>

<!-- Espacio donde se mostrará el valor del último botón presionado -->
<p id="output">Haz clic en un botón</p>


In [47]:
%%javascript
// Variables para almacenar el valor de cada botón
let valorBoton1, valorBoton2, valorBoton3;

// Asignar listeners a los botones
document.getElementById('btn1').addEventListener('click', function() {
    valorBoton1 = "Botón 1 presionado";
    document.getElementById('output').innerHTML = valorBoton1;
});

document.getElementById('btn2').addEventListener('click', function() {
    valorBoton2 = "Botón 2 presionado";
    document.getElementById('output').innerHTML = valorBoton2;
});

document.getElementById('btn3').addEventListener('click', function() {
    valorBoton3 = "Botón 3 presionado";
    document.getElementById('output').innerHTML = valorBoton3;
});


<IPython.core.display.Javascript object>

In [48]:
# Definir una función para manejar los datos enviados desde JavaScript
def manejar_click(valor):
    print(f"El valor recibido es: {valor}")

# Registrar la función con Colab para que sea invocable desde JavaScript
from google.colab import output
output.register_callback('notificar_click', manejar_click)


In [46]:
# https://docs.google.com/document/d/1Joeip5st2Dh8f1tpUizm_tgzyS3ggwLx/edit

In [49]:
%%html
<!-- Crear tres botones en HTML -->
<button id="btn1">Botón 1</button>
<button id="btn2">Botón 2</button>
<button id="btn3">Botón 3</button>

<!-- Espacio donde se mostrará el valor del último botón presionado -->
<h1 id="output">Haz clic en un botón</h1>

<script>
  let valorBoton1, valorBoton2, valorBoton3;

  // Asignar listeners a los botones
  document.getElementById('btn1').addEventListener('click', function() {
      valorBoton1 = "Botón 1 presionado";
      document.getElementById('output').innerHTML = valorBoton1;
  });

  document.getElementById('btn2').addEventListener('click', function() {
      valorBoton2 = "Botón 2 presionado";
      document.getElementById('output').innerHTML = valorBoton2;
  });

  document.getElementById('btn3').addEventListener('click', function() {
      valorBoton3 = "Botón 3 presionado";
      document.getElementById('output').innerHTML = valorBoton3;
  });
</script>
